In [ ]:
!unzip "/content/drive/MyDrive/Pytorch_practice/Cat_vs_dog/archive.zip"

Archive:  /content/drive/MyDrive/Pytorch_practice/Cat_vs_dog/archive.zip
  inflating: test/cats/cat_1.jpg     
  inflating: test/cats/cat_106.jpg   
  inflating: test/cats/cat_109.jpg   
  inflating: test/cats/cat_113.jpg   
  inflating: test/cats/cat_114.jpg   
  inflating: test/cats/cat_116.jpg   
  inflating: test/cats/cat_118.jpg   
  inflating: test/cats/cat_119.jpg   
  inflating: test/cats/cat_124.jpg   
  inflating: test/cats/cat_140.jpg   
  inflating: test/cats/cat_147.jpg   
  inflating: test/cats/cat_156.jpg   
  inflating: test/cats/cat_158.jpg   
  inflating: test/cats/cat_162.jpg   
  inflating: test/cats/cat_18.jpg    
  inflating: test/cats/cat_190.jpg   
  inflating: test/cats/cat_203.jpg   
  inflating: test/cats/cat_223.jpg   
  inflating: test/cats/cat_234.jpg   
  inflating: test/cats/cat_244.jpg   
  inflating: test/cats/cat_251.jpg   
  inflating: test/cats/cat_255.jpg   
  inflating: test/cats/cat_268.jpg   
  inflating: test/cats/cat_279.jpg   
  inflating: te

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.486, 0.456, 0.406], std=[0.299, 0.224, 0.225])

])

In [ ]:
train_dataset = ImageFolder(root = "/content/train", transform=transform)
test_dataset = ImageFolder(root = "/content/test", transform=transform)

In [ ]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 557
    Root location: /content/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=[0.486, 0.456, 0.406], std=[0.299, 0.224, 0.225])
           )

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
train_loader

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
      super(SimpleCNN, self).__init__()
      self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
      self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
      self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
      self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
      self.pool = nn.MaxPool2d(2, 2)

      self.fc1 = nn.Linear(256 * 4 * 4, 2048)
      self.fc2 = nn.Linear(2048, 2)


    def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = self.pool(F.relu(self.conv3(x)))
      x = self.pool(F.relu(self.conv4(x)))

      x = x.view(-1, 256 * 4 * 4)

      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x


model = SimpleCNN()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
  running_loss = 0.0
  for inputs, labels in train_loader:
    optimizer.zero_grad()
    # print("Inputs Resulst")
    # print(inputs)
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

Epoch 1, Loss: 0.6931314633952247
Epoch 2, Loss: 0.6715817583931817
Epoch 3, Loss: 0.6691781050629086
Epoch 4, Loss: 0.6609836485650804
Epoch 5, Loss: 0.6176047689384885
Epoch 6, Loss: 0.6034580336676704
Epoch 7, Loss: 0.5608530359135734
Epoch 8, Loss: 0.5124700715144476
Epoch 9, Loss: 0.4669964429405
Epoch 10, Loss: 0.4117017404900657
Epoch 11, Loss: 0.35582976539929706
Epoch 12, Loss: 0.3290947477022807
Epoch 13, Loss: 0.31662721849150127
Epoch 14, Loss: 0.2119154292676184
Epoch 15, Loss: 0.1610282998945978
Epoch 16, Loss: 0.12416185831858052
Epoch 17, Loss: 0.08360368799832132
Epoch 18, Loss: 0.05622652990536557
Epoch 19, Loss: 0.040534432149595685
Epoch 20, Loss: 0.03098469569037358


In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Accuracy of the network test images: {100 * correct // total} %')

Accuracy of the network test images: 66 %


In [ ]:
from google.colab import files
upload = files.upload()


filename = next(iter(upload))
print(f'Uploaded file : {filename}')

Saving pexels-svetozar-milashevich-1490908.jpg to pexels-svetozar-milashevich-1490908.jpg
Uploaded file : pexels-svetozar-milashevich-1490908.jpg


In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.299, 0.224, 0.225])
])

image = Image.open(filename).convert('RGB')


image = transform(image)

image = image.unsqueeze(0)


In [ ]:
model.eval()


with torch.no_grad():
  outputs = model(image)
  _, predicted = torch.max(outputs.data, 1)

classes = ['cat', 'dog']
print(f'Prediction : {classes[predicted.item()]}')

Prediction : dog
